# Football Database 
Football and Betting Statistics of the European Top5 Leagues

**D2TEC – Tecnologias de Big Data**<br />
Especialização em Ciência de Dados - IFSP Campinas

Alunos
- Daniel Vargas Shimamoto
- Diego Machado de Assis

# 0. Informações Gerais

## 0.1. Conhecendo  o dataset

Este conjunto de dados contém dados relacionados ao futebol contendo as cinco principais ligas da Europa entre os anos de 2014-2020. Os dados estão dividos em 7 tabelas e foram coletados das fontes https://understat.com/ e https://www.football-data.co.uk/

Fonte: https://www.kaggle.com/technika148/football-database 

## 0.2. Tabelas:

**Players**
 * PlayerID (Int64): Id dos jogadores (PK)
 * Player Name (Object): Nome dos Jogadores

**Teams**
 * teamID (int64): Id dos Times (PK)
 * name (object): Nome dos times

**Leagues** 
 * leagueID (int64): Id das Ligas (PK)
 * name (object): Nome das ligas
 * understatNotation (object): Nome da liga (em notação understat)

**teamstats**
 * gameID (int64): Id do jogo (PK,FK)
 * teamID (int64): Id dos Times (PK, FK)
 * season (int64): Temporada
 * date (object): Data
 * location (object): Jogo dentro ou fora 
 * goals (int64): Gols
 * XGoals (float64): Expectativa de gols
 * shots (int64): Chutes
 * shotsOnTarget (int64): Chutes no gol
 * deep (int64): Passes realizados a uma distância estimada de 20 jardas do gol (exceto cruzamento)
 * ppda (float64): Passes permitidos por ação defensiva no meio-campo adversário
 * fouls (int64): Faltas
 * corners (int64): Escanteios
 * yellowCard (int64): Cartões amarelos
 * redCards (int64): Cartões vermelhos
 * results (object): Resultado
  
**shots**
 * gameID (int64): Id do jogo (PK,FK)
 * shooterID (int64): Id dos jogadores que chutou(PK, FK)
 * assisterID (float64): Id dos jogadores que deu assistência (PK, FK)
 * minute (int64): minuto
 * situation (object): Situação de jogo
 * lastAction (object): Ultima ação antes de chutar
 * shotType (object): Tipo de chute
 * shotResult (object): Resultado do Chute
 * xGoal (float64): Expectativa de Gol
 * positionX (float64): Posição onde o chute foi feito (lado longo do campo)
 * positionY (float64): Posição onde o chute foi feito (lado curto do campo)

**appearences**
 * gameID (int64): Id do jogo (PK,FK)
 * playerID (int64): Id do Jogador (PK,FK)
 * goals (int64): Gols
 * ownGoals (int64): Gols contras
 * shots (int64): Chutes
 * xGoals (float64): Gols esperados
 * xGoalsChain (float64): Gols esperados de cada posse em que o jogador esteve envolvido
 * xGoalsBuildup (float64): XGoalsChain sem passes chaves e chutes
 * assists (int64): Assistências
 * keyPasses (int64): Passes chaves
 * xAssists (float64): Assistencias esperadas
 * position (object): Possição (
 * positionOrder (int64): Posição (Númerica)
 * yellowCard (int64): Cartões amarelos
 * redCards (int64): Cartões vermelhos
 * time (int64): Time
 * substituteIn (int64): Jogador que entrou
 * substituteOut (int64): Jogador que saiu
 * leagueID (int64): Id das Ligas (PK, FK)
 

**games**
 * leagueID (int64): Id das Ligas (PK, FK)
 * season (int64): Temporada
 * date (object): Data
 * homeTeamID (int64): Id do time mandante(PK, FK)
 * awayTeamID (int64): Id do time visitante (PK, FK)
 * homeGoals (int64): Gols marcados pelo time mandante
 * awayGoals (int64): Gols marcados pelo time visitante
 * homeProbability (float64): Probabilidade estimada de vitória do time mandante
 * drawProbability (float64): Probabilidade estimada de empate
 * awayProbability (float64): Probabilidade estimada de vitória do time visitante
 * homeGoalsHalfTime (int64): Gols marcados pelo time mandante até o intervalo
 * awayGoalsHalfTime (int64): Gols marcados pelo time visitante até o intervalo
 * B365H (float64): 
 * B365D (float64): 
 * B365A (float64): 
 * BWH (float64): 
 * BWD (float64): 
 * BWA (float64): 
 * IWH (float64): 
 * IWD (float64): 
 * IWA (float64): 
 * PSH (float64): 
 * PSD (float64): 
 * PSA (float64): 
 * WHH (float64): 
 * WHD (float64): 
 * WHA (float64): 
 * VCH (float64): 
 * VCD (float64): 
 * VCA (float64): 
 * PSCH (float64): 
 * PSCD (float64): 
 * PSCA (float64): 


![](../assets/FootballDatabase.model.png)

In [1]:
# import sqlite3
# import pandas as pd
# conn = sqlite3.connect('FootballDb.db')

In [2]:
!pip install mysql-connector-python

In [3]:
import mysql.connector
from mysql.connector import errorcode

DB_CONFIG = {
    'user': 'd2tec_readonly',
    'password': 'L7pH!yVpXCXzy#p0FDiCg0h2',
    'host': 'd2tec-trabalho1-football-database.cfwkca29qevn.us-east-1.rds.amazonaws.com',
    'database': 'football',
    'raise_on_warnings': True
}

In [4]:
def mysql_connect(config):
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print(f'[CONNECTION ERROR] Invalid credentials: {err}')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print(f'[CONNECTION ERROR] Database does not exist: {err}')
        else:
            print(f'[CONNECTION ERROR] {err}')
        return (None, None)
    else:
        cursor = cnx.cursor()
        return (cnx, cursor)
    
def mysql_close_connection(cnx):
    try:
        cnx.close()
    except mysql.connector.Error as err:
        print(f'[CLOSECONNECTION ERROR] Failes closing connection to mysql: {err}')
        raise

In [5]:
import pandas as pd

def exec_query(query):
    cnx, cursor = mysql_connect(DB_CONFIG)
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=cursor.column_names)
    mysql_close_connection(cnx)
    return df

# Perguntas

## 1) Qual o time com percentual de vitórias?

## 2) Qual time que tinha maior probabilidade de vitórias nos jogos

## 3) Qual jogador fez mais gols?

## 4) Qual jogador fez mais assistencias?

## 5) Qual posição do campo mais sai gols?

## 6) Times que jogam como mandante vencem mais?

## 7) Times com jogadores expulsos perdem mais?

## 8) As expectativas de gols acertam?

## 9) Como é a distribuição dos times entre as ligas?

## 1) Qual o time com percentual de vitórias?

In [6]:
question_1 = '''
SELECT
    name as Team,
    sum(vitorias) as Wins,
    count(vitorias) as Games,
    100 * cast(sum(vitorias) as float)/cast(count(vitorias) as float) as Win_rate
FROM(
    SELECT 
    CASE 
        WHEN results = 'W' THEN 1 
            ELSE 0 
        END AS vitorias,
    name
    FROM teamstat
    LEFT JOIN team ON teamstat.teamID = team.teamID
    ) AS v
GROUP BY name
ORDER BY Win_rate DESC
'''

exec_query(question_1)

,Team,Wins,Games,Win_rate
0,Bayern Munich,179,238,75.210084
1,Paris Saint Germain,187,255,73.333333
2,Juventus,191,266,71.804511
3,Barcelona,190,266,71.428571
4,Manchester City,183,266,68.796992
...,...,...,...,...
141,Middlesbrough,5,38,13.157895
142,Cesena,4,38,10.526316
143,Nuernberg,3,34,8.823529
144,Cordoba,3,38,7.894737


## 2) Quais são os confrontos que o time visitante tem a maior probabilidade estimada média de ganhar?

In [7]:
question_2 = '''
SELECT 
    A.name as homeTeam,
    team.name as awayTeam,
    avg(homeProbability) avg_homeProbability,
    avg(drawProbability) avg_drawProbability,
    avg(awayProbability) avg_awayProbability
FROM
    (
    SELECT
        name,
        awayTeamID,
        homeProbability,
        drawProbability,
        awayProbability
    FROM
        game
    LEFT JOIN team ON game.homeTeamID = team.teamID
    ) A
LEFT JOIN team ON A.awayTeamID = team.teamID
GROUP BY
    A.name,
    team.name
ORDER BY
    avg_awayProbability DESC
'''
exec_query(question_2)

,homeTeam,awayTeam,avg_homeProbability,avg_drawProbability,avg_awayProbability
0,Cordoba,Barcelona,0.0002,0.00120,0.99860
1,Lecce,Atalanta,0.0005,0.00170,0.99780
2,Spezia,Inter,0.0004,0.02240,0.97720
3,Elche,Barcelona,0.0028,0.02595,0.97125
4,Nancy,Monaco,0.0035,0.02670,0.96980
...,...,...,...,...,...
3693,Monaco,Nancy,0.9934,0.00620,0.00040
3694,Juventus,Cesena,0.9454,0.05430,0.00030
3695,Torino,Cesena,0.9934,0.00640,0.00020
3696,Huddersfield,Swansea,0.7666,0.23340,0.00000


## 3) Qual jogador fez mais gols e qual sua taxa de gols por chute

In [9]:
question_3 = '''
SELECT
    A.*,
    100 * CAST(goals AS FLOAT) / CAST(shots AS FLOAT) AS goal_rate
FROM(
    SELECT 
        player.name as player,
        COUNT(shotResult) as shots,
        SUM(
            CASE 
            WHEN shotResult = 'Goal' 
                THEN 1 
                ELSE 0 
            END) AS goals
    FROM
        shot
    LEFT JOIN player on shot.shooterID = player.playerID
    GROUP BY 
        player.name
    ) A
ORDER BY 
    goals DESC

'''
exec_query(question_3)

,player,shots,goals,goal_rate
0,Lionel Messi,1392,272,19.540230
1,Cristiano Ronaldo,1463,241,16.473001
2,Robert Lewandowski,974,207,21.252567
3,Luis Suárez,811,176,21.701603
4,Harry Kane,889,163,18.335208
...,...,...,...,...
5257,Jacques Siwe,3,0,0.000000
5258,Saidou Sow,2,0,0.000000
5259,Yanis Lhéry,1,0,0.000000
5260,Wilitty Younoussa,2,0,0.000000


## 4) Qual jogador participou de mais gols (Assistencia + Gol) ?

In [10]:
question_4 = '''
WITH 
t_goals AS(
    SELECT
        player.name as player,
        SUM(
            CASE 
            WHEN shotResult = 'Goal' THEN 1 
                ELSE 0 
            END) AS goals
    FROM
        shot
    LEFT JOIN player on shot.shooterID = player.playerID
    GROUP BY player.name
    ),

t_assistence AS(
    SELECT
        player.name as assist_player,
        SUM(
            CASE 
            WHEN shotResult = 'Goal' THEN 1 
                ELSE 0 
            END) AS assistence
    FROM
        shot
    LEFT JOIN player on shot.assisterID = player.playerID

    GROUP BY 
        player.name
    )

SELECT
    t_assistence.assist_player player,
    t_assistence.assistence,
    t_goals.goals,
    t_assistence.assistence + t_goals.goals as total_participation
FROM
    t_assistence
LEFT JOIN 
    t_goals on t_assistence.assist_player = t_goals.player

WHERE 
    player <> 'None'
ORDER BY
    total_participation DESC

'''
exec_query(question_4)

,player,assistence,goals,total_participation
0,Lionel Messi,97,272,369
1,Luis Suárez,146,176,322
2,Cristiano Ronaldo,54,241,295
3,Robert Lewandowski,32,207,239
4,Harry Kane,34,163,197
...,...,...,...,...
4908,Franco Zuculini,0,0,0
4909,Felipe Avenatti,0,0,0
4910,Simone Calvano,0,0,0
4911,Enrico Bearzotti,0,0,0


## 5) Times que jogam como mandante vencem mais?


In [11]:
question_5 = '''
with t_win as(
SELECT
    name,
    CASE 
        WHEN homeGoals > awayGoals THEN 'homeWin'
        WHEN homeGoals < awayGoals THEN 'awayWin'
        WHEN homeGoals = awayGoals THEN 'draw'
        ELSE 'not_a_result'
    END as Wining
FROM
    game
LEFT JOIN
    team on game.awayTeamID = team.teamID
)
SELECT
    name,
    count(Wining) as awayVictoryTotal
FROM
    t_win
WHERE
    Wining ='awayWin'
GROUP BY
    name
ORDER BY
    awayVictoryTotal DESC
'''
exec_query(question_5)

,name,awayVictoryTotal
0,Barcelona,85
1,Paris Saint Germain,85
2,Manchester City,84
3,Bayern Munich,84
4,Real Madrid,82
...,...,...
139,Queens Park Rangers,2
140,Cordoba,2
141,Middlesbrough,1
142,Cesena,1


## 6) Qual time fez mais gols em uma única temporada?

In [13]:
question_6 = '''
SELECT
    team.name AS team,
    season,
    SUM(goals) AS goals,
    COUNT(*) AS games,
    CAST(SUM(goals) AS FLOAT)/CAST(count(*) AS FLOAT) AS goals_per_game
FROM
    teamstat
LEFT JOIN 
    team ON teamstat.teamID = team.teamID
GROUP BY
    team,
    season
ORDER BY
    goals DESC
'''
exec_query(question_6)

,team,season,goals,games,goals_per_game
0,Real Madrid,2014,118,38,3.105263
1,Barcelona,2016,116,38,3.052632
2,Barcelona,2015,112,38,2.947368
3,Real Madrid,2015,110,38,2.894737
4,Barcelona,2014,110,38,2.894737
...,...,...,...,...,...
681,Malaga,2017,24,38,0.631579
682,Cordoba,2014,22,38,0.578947
683,Huddersfield,2018,22,38,0.578947
684,Toulouse,2019,22,28,0.785714


## 7) Quais jogadores tomaram mais cartões (Amarelo + Vermelho)## 7) As expectativas de gols acertam?

In [14]:
question_7 = '''
SELECT
    name,
    sum(redCards) as totalRedCard,
    sum(yellowCard) as totalYellowCard,
    sum(redCards) + sum(yellowCard) as totalCard
FROM
    appearance
LEFT JOIN player ON appearance.playerID = player.playerID
GROUP BY
    name
ORDER BY
    totalCard DESC
'''
exec_query(question_7).head()

,name,totalRedCard,totalYellowCard,totalCard
0,Danilo,6,83,89
1,Raúl García,2,81,83
2,Dani García,2,80,82
3,Álvaro González,4,74,78
4,Damián Suárez,4,71,75


## 8) Como é a distribuição dos times entre as ligas?